In [19]:
import duckdb


In [20]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [25]:
df = con.execute("""
    select * from(
            select *, row_number() over (partition by NATBR order by data_ingestao desc) as row_num 
                from bronze_z0019
                where data_ingestao >= '2025-07-14'                 
        )
    where row_num = 1
                 """).fetchdf()

In [28]:
df_final = df.drop(columns=['nome_arquivo', 'data_ingestao', 'row_num'])
df_final = df_final.rename(columns={'NATBR': 'id'})
df_final = df_final.rename(columns={'MAKTX': 'nm_produto'})
df_final = df_final.rename(columns={'WERKS': 'id_categoria'})
df_final = df_final.rename(columns={'MAINS': 'id_fornecedor'})
df_final = df_final.rename(columns={'LABST': 'vl_preco'})
df_final.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200
1,10002,MARTELO,BT50,100,1500
2,10003,PREGO,BT10,100,60
3,10005,MACHADO,BT50,100,100
4,10001,PARAFUSO,BT10,100,100


In [31]:
df2 = df_final
df2 = df2.astype({
        'id': int,
        'nm_produto': str,
        'id_categoria': str,
        'id_fornecedor': int,   
        'vl_preco': float
        })
df2.dtypes

id                 int64
nm_produto        object
id_categoria      object
id_fornecedor      int64
vl_preco         float64
dtype: object

In [34]:
con.execute("""
create table if not exists produtos
            (
            id bigint,
            nm_produto text,
            id_categoria text,
            id_fornecedor bigint,
            vl_preco float
            )
""")

In [35]:
df2.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10002,MARTELO,BT50,100,1500.0
2,10003,PREGO,BT10,100,60.0
3,10005,MACHADO,BT50,100,100.0
4,10001,PARAFUSO,BT10,100,100.0


In [37]:
con.execute("insert into produtos select * from df2")

In [38]:
df_resultado = con.execute("select * from produtos").fetchdf()
df_resultado.head(10)

,id,nm_produto,id_categoria,id_fornecedor,vl_preco
0,10004,SERRA,BT50,100,200.0
1,10002,MARTELO,BT50,100,1500.0
2,10003,PREGO,BT10,100,60.0
3,10005,MACHADO,BT50,100,100.0
4,10001,PARAFUSO,BT10,100,100.0
